In [1]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


# 1. Naives Bayes Theorem

Flow:

1. Read data:
```py
positive = open("positive.txt", "r").read()
negative = open("negative.txt", "r").read()
```

2. Tokenize datanya
Karena data yang diread akan dipindahkan ke dalam variable dalam tipe String. Kita harus tokenize dulu untuk bagian pre-processing datanya.
```py
list_words = word_tokenize(positive) + word_tokenize(negative)
```

3. Bersihin kata-kata

- Stopwords
```py
list_words = [word for word in list_words if word.lower() not in eng_stopwords]
```
- Punctuation
```py
list_words = [word for word in list_words if word.lower() not in string.punctuation]
```
- Ngilangin angka
```py
list_words = [word for word in list_words if word.isalpha()] # append ke list_words kalau word adalah sebuah alphabet
```

4. Labeling sentences (pisahin kategori positif & negatif reviews)
```py
labeled_sentence = []
for sentence in positive.split("\n"):
    labeled_sentence.append((sentence, "pos"))

for sentence in negative.split("\n"):
    labeled_sentence.append((sentence, "neg"))
```

5. Membuat dataset (dari kata2 + labeled_sentence yang udah kita buat)
```py
dataset = []
for sentence, label in labeled_sentence:
    dict = {}
    # word tokenize sentencesnya
    words = word_tokenize()
    # masukin ke dalam dictionarynya
    for feature in list_words:
        key = feature # kata
        value = feature in words # kita mau cek aja sih ada ga sih kata-kata dalam kalimat kita yang ada dalam list of words bersih yang udah kita buat.
        dict[key] = value # contoh {"love" : False, "damn" : True}
    # kalau dictionarynya udah dibuat, kita masukin dia ke dataset yg udah kita buat
    dataset.append((dict, label))